In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
from pprint import pprint
import json
# from icecream import ic
import azure.functions as func
import logging
from json2html import json2html
from openpyxl import Workbook, styles, load_workbook
from io import StringIO, BytesIO
import base64
from tempfile import NamedTemporaryFile


def save_virtual_workbook(workbook):
    """Save an openpyxl workbook in memory."""

    try:
        # Save workbook to a temporary file and read its content
        with NamedTemporaryFile() as f:
            workbook.save(f.name)
            # self.logger.debug("Workbook saved to a temporary file.")
            f.seek(0)
            content = f.read()
            # self.logger.debug("Workbook read into memory.")
        return content

    except Exception as e:  # noqa: e is intentionally unused
        # self.logger.error(f"Error in save_virtual_workbook: {e}")
        raise


# Add the project root directory to sys.path

os.environ['KEY_VAULT_NAME'] = 'devintaccountingftkeys'
settings_file = "../hrmlib/devint_settings.yaml"

In [3]:
from hrmlib.hrmtools import (
    SecretsAndSettingsManager, HTMLListHandler,
    read_html_page_template,
    extract_data_from_received_http_request,
    replace_and_format_html_template,
    DevIntConnector
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# Check if the StreamHandler is already added
if not any(isinstance(handler, logging.StreamHandler) for handler in logger.handlers):
    print_log_handler = logging.StreamHandler()  # StreamHandler prints to console
    print_log_handler.setLevel(logging.INFO)
    logger.addHandler(print_log_handler)

dc = DevIntConnector(parent_logger=logger,
                     settings_file=settings_file)
config = SecretsAndSettingsManager(parent_logger=logger)
dc.setup(config)
selectors = []
selectors.append({'type': 'request', 'condition': 'devtest'})
instructions = dc.read_instruction_files()
bookings = dc.get_all_bb_posts()
asset_stock_accounts = dc.get_all_bb_accounts()
expected_bookings = dc.read_expected_bookings()
reports = dc.build_reports(
    bookings, expected_bookings, instructions, asset_stock_accounts)

Total instruction files processed: 5
All time slots processed successfully.
Found 1 'Kontenrahmen' sheets.
Completed processing all dataframes for 'kontenrahmen'.
Found 'Kostenstellenplan' sheet.
Dropped empty rows. Remaining rows: 124
Starting to process each row in the dataframe.
Completed processing kostenstellenplan.
Found 'Reports Overview' sheet.
All time slots processed successfully.
Found 1 'Kontenrahmen' sheets.
Completed processing all dataframes for 'kontenrahmen'.
Found 'Kostenstellenplan' sheet.
Dropped empty rows. Remaining rows: 124
Starting to process each row in the dataframe.
Completed processing kostenstellenplan.
Found 'Reports Overview' sheet.
4 report plans were not read because of their deprecated version.
Processing reports for budget plans.
Found 'Flow Plan' sheet.
Completed reading instruction files.
Starting to build personnel bookings DataFrame.
Error processing 'transaction_id_by_customer' in booking at row 2: boolean value of NA is ambiguous
Error processi

In [4]:
wb = load_workbook(BytesIO(reports['allSheetsFile']))
attachment_content = base64.b64encode(
    save_virtual_workbook(wb)).decode()
# save workbook to file for debugging: "output.xlsx"
with open("output.xlsx", "wb") as output:
    output.write(base64.b64decode(attachment_content))